In [1]:
import pandas as pd
import sqlite3
import datetime
pd.set_option('display.max_columns', None)
cnx = sqlite3.connect('results.db')

## Janowiec Wielkopolski

In [2]:
results = []
with open('janowiec-wielkopolski.txt') as f:
    for line in f.readlines():
        parts = line.split('\t')
        results.append([parts[5] + ' ' + parts[7], parts[9], (parts[15] or parts[14])[:1], (parts[19] or parts[18])[:3], parts[11] or None, int(next(p for p in parts if len(p.strip()) == 4 and p[:2] in {'19', '20'})), parts[-1].strip() or parts[-2].strip()])

janowiec = pd.DataFrame(results, columns=[
    'name',
    'city',
    'gender',
    'category',
    'team',
    'year_of_birth',
    'result',
])
assert not any(janowiec['name'].duplicated())
# sorted(janowiec[janowiec['team'].notnull()]['team'].unique())
janowiec.loc[janowiec['team'] == 'TRACK GANG', 'team'] = 'TRACK GANG TORUŃ'
janowiec.loc[janowiec['team'] == 'WATAHA ŁĄCZY NAS BIEGANIE', 'team'] = 'WATAHA - ŁĄCZY NAS BIEGANIE'
janowiec.loc[janowiec['team'] == 'EKO- OPAŁ PŁONNE', 'team'] = 'EKO-OPAŁ PŁONNE'

janowiec

,name,city,gender,category,team,year_of_birth,result
0,ABRAMOWSKI DAMIAN,ŻNIN,M,M30,TT SODA,1991,00:37:45
1,ADAMSKA IZABELA,JANOWIEC WIELKOPOLSKI,K,K40,None,1985,00:47:33
2,ADAMSKA ANNA,KŁECKO,K,K40,LECHITA BIEGA KŁECKO,1985,00:57:09
3,ADAMSKI ADAM,ŻNIN,M,M30,None,1986,00:47:48
4,ANGIEL WŁADYSŁAW,BYDGOSZCZ,M,M80,None,1937,01:40:32
...,...,...,...,...,...,...,...
267,ZMUDZIŃSKA JOANNA,PIECHCIN,K,K40,None,1983,00:45:37
268,ŚWIĘCICKI ANDRZEJ,BYDGOSZCZ,M,M50,BYDGOSKIE ŻÓŁWIE,1973,00:50:43
269,ŁASIŃSKI DARIUSZ,KOWALEWO,M,M30,None,1991,00:44:20
270,ŻUCHOWSKI JÓZEF,PŁONNE,M,M50,EKO-OPAŁ PŁONNE,1971,00:41:23


## Fabianki

In [3]:
fabianki = pd.read_csv('fabianki.txt', sep='\t', names=['c0', 'c1', 'name', 'gender', 'c2', 'c3', 'category', 'c5', 'team', 'c6', 'c7', 'c8', 'c9', 'result', 'c11'])
fabianki = fabianki[['name', 'gender', 'category', 'team', 'result']]
fabianki['name'] = fabianki['name'].str.upper()
fabianki['team'] = fabianki['team'].str.upper()
fabianki = fabianki[fabianki['result'].notnull()]
fabianki['category'] = fabianki['category'].str.slice(0, 3)

fabianki.loc[fabianki['team'] == 'PAŁUKI RUNNIG TEAM', 'team'] = 'PAŁUKI RUNNING TEAM'
fabianki.loc[fabianki['team'] == 'T G SOKÓŁ GNIEWKOWO', 'team'] = 'TG SOKÓŁ GNIEWKOWO'
fabianki.loc[fabianki['team'] == 'T.G.SOKÓŁ GNIEWKOWO', 'team'] = 'TG SOKÓŁ GNIEWKOWO'
fabianki.loc[fabianki['team'] == 'WKB MARATONCZYK WLOCLAWEK', 'team'] = 'WKB MARATOŃCZYK WŁOCŁAWEK'
fabianki.loc[fabianki['team'] == 'WKB MARATONCZYK WŁOCŁAWEK', 'team'] = 'WKB MARATOŃCZYK WŁOCŁAWEK'

any(fabianki['name'].duplicated())

fabianki

,name,gender,category,team,result
0,KLEDZIK ZBIGNIEW,M,M30,PAŁUKI RUNNING TEAM,00:32:21
1,TOMASZEWSKI MIŁOSZ,M,M30,GKS FABIANKI,00:33:05
2,TOMASZEWSKI MACIEJ,M,M30,TASMAN TEAM,00:33:46
3,KUNECKI DARIUSZ,M,M40,KUNA TEAM WŁOCŁAWEK,00:34:42
4,SZTOLCMAN PIOTR,M,M30,GKS FABIANKI,00:34:55
...,...,...,...,...,...
161,GLAZIK RENATA,K,K50,FABIANKI,01:18:26
162,WITKOWSKA EDYTA,K,K50,FABIANKI,01:18:32
163,WITKOWSKA JULIA,K,K16,FABIANKI,01:18:33
164,ANDELBRAT BEATA,K,K50,GRUDZIADZ,01:18:33


## Merge

In [4]:
after_2 = janowiec.merge(fabianki, on=['name', 'gender'], how='outer', suffixes=('_janowiec', '_fabianki'))

In [5]:
after_2[after_2['team_janowiec'].notnull() & after_2['team_fabianki'].notnull() & (after_2['team_janowiec'] != after_2['team_fabianki'])]

,name,city,gender,category_janowiec,team_janowiec,year_of_birth,result_janowiec,category_fabianki,team_fabianki,result_fabianki
72,GRABARSKA MAGDALENA,BRZOZÓWKA,K,K30,TRACK GANG TORUŃ,1986.0,00:47:21,K30,BRZOZÓWKA,00:45:24
118,KARPIŃSKI PAWEŁ,GOLUB-DOBRZYŃ,M,M20,ZAKOLATEAMFAMILIA,1996.0,00:39:14,M16,#ZAKOLATEAM,00:38:35
321,TOMASZEWSKI WOJCIECH,TORUŃ,M,M60,START TEAM WĄBRZEŹNO,1964.0,00:41:14,M60,TASMAN TEAM,00:39:40


In [6]:
after_2[after_2['category_janowiec'].notnull() & after_2['category_fabianki'].notnull() & (after_2['category_janowiec'] != after_2['category_fabianki'])]

,name,city,gender,category_janowiec,team_janowiec,year_of_birth,result_janowiec,category_fabianki,team_fabianki,result_fabianki
118,KARPIŃSKI PAWEŁ,GOLUB-DOBRZYŃ,M,M20,ZAKOLATEAMFAMILIA,1996.0,00:39:14,M16,#ZAKOLATEAM,00:38:35
235,PIETRZAK MICHAŁ,WŁOCŁAWEK,M,M20,KUNA TEAM WŁOCŁAWEK,2005.0,00:36:42,M16,KUNA TEAM WŁOCŁAWEK,00:36:28


In [7]:
after_2['team'] = after_2['team_fabianki'].combine_first(after_2['team_janowiec'])
after_2['category'] = after_2['category_fabianki'].combine_first(after_2['category_janowiec'])
after_2 = after_2.drop(labels=['category_janowiec', 'team_janowiec', 'category_fabianki', 'team_fabianki'], axis=1)

In [8]:
after_2

,name,city,gender,year_of_birth,result_janowiec,result_fabianki,team,category
0,ABRAMOWSKI DAMIAN,ŻNIN,M,1991.0,00:37:45,NaN,TT SODA,M30
1,ADAMSKA ANNA,KŁECKO,K,1985.0,00:57:09,NaN,LECHITA BIEGA KŁECKO,K40
2,ADAMSKA IZABELA,JANOWIEC WIELKOPOLSKI,K,1985.0,00:47:33,NaN,None,K40
3,ADAMSKI ADAM,ŻNIN,M,1986.0,00:47:48,NaN,None,M30
4,ANDELBRAT BEATA,NaN,K,NaN,NaN,01:18:33,GRUDZIADZ,K50
...,...,...,...,...,...,...,...,...
372,ŁASIŃSKI DARIUSZ,KOWALEWO,M,1991.0,00:44:20,NaN,None,M30
373,ŚWIĘCICKI ANDRZEJ,BYDGOSZCZ,M,1973.0,00:50:43,NaN,BYDGOSKIE ŻÓŁWIE,M50
374,ŻUCHOWSKI BARTOSZ,NaN,M,NaN,NaN,00:37:34,PŁONNE,M16
375,ŻUCHOWSKI JÓZEF,PŁONNE,M,1971.0,00:41:23,00:40:08,EKO-OPAŁ PŁONNE,M50


In [9]:
after_2[after_2['team'] == 'WKB MARATONCZYK WŁOCŁAWEK']

,name,city,gender,year_of_birth,result_janowiec,result_fabianki,team,category


In [10]:
sorted(after_2[after_2['team'].notnull()]['team'].unique())

['#SLIMAKIBARCIN',
 '#WKTM',
 '#ZAKOLATEAM',
 'ALTOM GNIEZNO',
 'AMATEUR RUNNERS WŁOCŁAWEK',
 'BACHORZEWO',
 'BANALNY TRENER',
 'BANICI ULTRA ADVENTURE',
 'BBL INOWROCŁAW',
 'BENEDEK-TEAM',
 'BIEGOWI PRZYJACIELE',
 'BOBROWNIKI',
 'BRZOZÓWKA',
 'BYDGOSKIE ŻÓŁWIE',
 'BYDGOSZCZ',
 'BĄDKOWO',
 'CHOCEŃ',
 'CYBORGI',
 'DECATHLON TEAM INOWROCŁAW',
 'DOBRZYŃ NAD WISŁĄ',
 'DREAM TEAM CHAWŁODNO',
 'EKO-OPAŁ PŁONNE',
 'ERKA RUN TEAM',
 'FABIANKI',
 'FUNDACJA SODA',
 'GDAŃSK',
 'GKS FABIANKI',
 'GRUDZIADZ',
 'JACKASS',
 'JANOWY MŁYN',
 'KARNKOWO',
 'KB MANIAC POZNAŃ',
 'KLECZEW',
 'KORNELIUSZ SERWIS KOMPUTEROWY',
 'KROJCZYN',
 'KS LZS KRUSZWICA',
 'KS SOKÓŁ GĄSAWA',
 'KS SZOSA ŻNIN',
 'KUJAWSKO-POMORSKIE',
 'KUNA TEAM WŁOCŁAWEK',
 'LECHITA BIEGA KŁECKO',
 'LINGUA PRO BYDGOSZCZ',
 'LIPNO',
 'LKS SPARTA JANOWIEC WLKP',
 'LZS JASTRZĘBIE',
 'MALINOWSCY MIGLASST TEAM',
 'MAŁKINIA GÓRNA',
 'NAKŁO NAD NOTECIĄ',
 'NOWA WIEŚ',
 'OSTROWSKI TEAM ŻNIN',
 'PACIFIC NESTLE',
 'PARKRUN TORUŃ',
 'PAŁUKI RUNNING TE

## Save

In [11]:
def to_seconds(time_str):
    t = datetime.time.fromisoformat(time_str)
    return t.hour * 3600 + t.minute * 60 + t.second

def to_time_str(seconds):
    return datetime.datetime.fromtimestamp(seconds, datetime.UTC).strftime('%T')
    
def get_result(row):
    results = [row[col] for col in row.index if col.startswith('result_') and type(row[col]) == str]
    row['total_runs'] = len(results)
    best_6_runs = sorted([to_seconds(r) for r in results])[-6:]
    row['avg_result'] = sum(best_6_runs) / len(best_6_runs)
    row['avg_result_str'] = to_time_str(row['avg_result'])
    return row

In [14]:
gp = after_2.apply(get_result, axis=1)
gp['year_of_birth'] = gp['year_of_birth'].astype('Int64')
gp

,name,city,gender,year_of_birth,result_janowiec,result_fabianki,team,category,total_runs,avg_result,avg_result_str
0,ABRAMOWSKI DAMIAN,ŻNIN,M,1991,00:37:45,NaN,TT SODA,M30,1,2265.0,00:37:45
1,ADAMSKA ANNA,KŁECKO,K,1985,00:57:09,NaN,LECHITA BIEGA KŁECKO,K40,1,3429.0,00:57:09
2,ADAMSKA IZABELA,JANOWIEC WIELKOPOLSKI,K,1985,00:47:33,NaN,None,K40,1,2853.0,00:47:33
3,ADAMSKI ADAM,ŻNIN,M,1986,00:47:48,NaN,None,M30,1,2868.0,00:47:48
4,ANDELBRAT BEATA,NaN,K,<NA>,NaN,01:18:33,GRUDZIADZ,K50,1,4713.0,01:18:33
...,...,...,...,...,...,...,...,...,...,...,...
372,ŁASIŃSKI DARIUSZ,KOWALEWO,M,1991,00:44:20,NaN,None,M30,1,2660.0,00:44:20
373,ŚWIĘCICKI ANDRZEJ,BYDGOSZCZ,M,1973,00:50:43,NaN,BYDGOSKIE ŻÓŁWIE,M50,1,3043.0,00:50:43
374,ŻUCHOWSKI BARTOSZ,NaN,M,<NA>,NaN,00:37:34,PŁONNE,M16,1,2254.0,00:37:34
375,ŻUCHOWSKI JÓZEF,PŁONNE,M,1971,00:41:23,00:40:08,EKO-OPAŁ PŁONNE,M50,2,2445.5,00:40:45


In [15]:
gp.to_sql(name='results', con=cnx, if_exists='replace', index_label='id')

377